# Introductory Tutorial

In this tutorial, you will:

* Add a simple workflow to the central database via a file

* Run that workflow in a few modes

* Get a flavor of the Python API

The purpose of this tutorial is to get you set up as quickly as possible; it isn’t intended to demonstrate the features of FireWorks or explain things in great detail. This tutorial can be safely completed from the command line, and requires no programming.

First, reset again the FireWorks database with ```lpad reset```.



## Add a Firework to the LaunchPad

A Firework contains a list of computing tasks (Firetasks) to be performed. For this tutorial, we will use a Firework that consists of only a single step. We’ll tackle more complex workflows in other tutorials. Our workflow consisting of one Firework and one Firetask thus looks like this:

![firetask](images/single_fw.png)

Let's add the firework to the database of jobs:

In [ ]:
!lpad add fw_test.yaml

This command added a simple workflow to the database which was serialized into a file called fw_test.yaml. This workflow is just a single step that print some text to a file. Look inside fw_test.yaml with a text editor to see how that workflow was defined:

In [ ]:
!cat fw_test.yaml

You should have received confirmation that the Firework got added. You can query the database for this Firework as follows:

In [ ]:
!lpad get_fws -i 1 -d all

This prints, in JSON format, all details of the Firework with fw_id = 1 (the first Firework entered into the database):

Some of the Firework is straightforward, but a few sections deserve further explanation:

The spec of the Firework contains all the information about what job to run and the parameters needed to run it.

Within the spec, the _tasks section tells you what jobs will run. The ScriptTask is a particular type of task that runs commands through the shell. Other sections of the spec can be also be defined, but for now we’ll stick to just _tasks. Later on, we’ll describe how to run multiple _tasks or customized _tasks.

This Firework runs the script echo "howdy, your job launched successfully!" >> howdy.txt", which prints text to a file named howdy.txt.

The state of READY means the Firework is ready to be run.

The name is an optional field that we can set to help query for FireWorks later on. In this case, we did not specify one so a default name was used.
You have now stored a Firework in the LaunchPad, and it’s ready to run!

## Launch jobs

We can launch jobs using ```rlaunch``` ("Rocket" launch) command:

In [ ]:
!rlaunch singleshot

This command fetches an available Firework from the FireWorks database and runs it.

Verify that the desired task ran:

In [ ]:
!cat howdy.txt

You should see the text: "howdy, your job launched successfully!"

In addition to howdy.txt, you should also see a file called FW.json. This contains a JSON representation of the Firework that the Rocket ran and can be useful later for tracking down a launch or debugging.

Check the status of your Firework:

In [ ]:
!lpad get_fws -i 1 -d all

You will now see lots of information about your Rocket launch, such as the time and directory of the launch. A lot of it is probably unclear, but you should notice that the state of the Firework is now COMPLETED.

Try launching another rocket:

In [ ]:
!rlaunch singleshot

The message "No FireWorks are ready to run and match query!" indicates that it tried to fetch a Firework from the database, but none could be found. Indeed, we had previously run the only Firework that was in the database.

## Launch many Rockets (rapidfire mode)

If you just want to run many jobs on the central server itself, the simplest way is to run the Rocket Launcher in “rapidfire mode”. Let’s try this feature:

Let’s add a Fireworks 3 times:

In [ ]:
!lpad add fw_test.yaml

Confirm that the three Fireworks got added to the database, in addition to the one from before (4 total):

In [ ]:
!lpad get_fws -d less

We could also just get information for jobs that are ready to run (our 3 new FireWorks):

In [ ]:
!lpad get_fws -s READY -d less

Let’s launch jobs in “rapidfire” mode, which will keep repeating until we run out of Fireworks to run:

In [ ]:
!rlaunch rapidfire

You should see three directories starting with the tag launcher_. Inside each of these directories, you’ll find the results of one of your FireWorks (a file named howdy.txt):

In [ ]:
!cat launch*/howdy.txt

## Running FireWorks automatically

We can set our Launcher to continuously look for new FireWorks to run. Let’s try this feature.

Start the Launcher in a terminal so that it looks for new FireWorks every 10 seconds:

```rlaunch rapidfire --nlaunches infinite --sleep 10```

Let’s insert two FireWorks:

In [ ]:
!lpad add fw_test.yaml

After a few seconds, the Rocket Launcher should have picked up the new jobs and run them. Confirm this is the case:

In [ ]:
!cat launch*/howdy.txt

You should see the outputs for each Firework we inserted.

You can continue adding FireWorks as desired; the Launcher will run them automatically and create a new directory for each job.

As with all FireWorks scripts, you can run the built-in help for more information:

```rlaunch -h
rlaunch singleshot -h
rlaunch rapidfire -h```

## What just happened?

It’s important to understand that when you add a Firework to the LaunchPad using the lpad script, the job just sits in the database and waits. The LaunchPad does not submit jobs to a computing resource when a new Firework is added to the LaunchPad. Rather, a computing resource must request a computing task by running the Launcher.

By running the Launcher from different locations, you can have different computing resources run your jobs. Using rapidfire mode is a convenient way of requesting multiple jobs using a single command.


## Python Examples

While it’s possible to work operate FireWorks using YAML or JSON files, a much cleaner mode of operation is to use Python scripts. For example, here is a runnable script that creates our LaunchPad, defines our test Workflow, and runs it:

In [ ]:
from fireworks import Firework, LaunchPad, ScriptTask
from fireworks.core.rocket_launcher import launch_rocket

# set up the LaunchPad and reset it
launchpad = LaunchPad.auto_load()
launchpad.reset('', require_password=False)

# create the Firework consisting of a single task
firetask = ScriptTask.from_str('echo "howdy, your job launched successfully!"')
firework = Firework(firetask)

# store workflow and launch it locally
launchpad.add_wf(firework)
launch_rocket(launchpad)

## Summary

At this point, you’ve successfully stored a simple job in a database and run it later on command. You even executed multiple jobs with a single command: rlaunch rapidfire, and looked for new jobs automatically using the infinite mode. This should give a basic feeling of how you can automate many jobs using FireWorks.

However, we still haven’t covered many important topics. For example, we have not executed complex workflows (and in particular materials science workflows), run arbitrary Python code, or run jobs on different types of computing resources.